**Notice**: this notebook has been used to write the code easily. The notebook has then been [properly modified and] converted into a python file, "../src/generate_comparison_data.py", that has then been executed on an aws ec2 machine.

In [1]:
import sys
sys.path.insert(0, '../src')
import numpy as np

from frederickson import Frederickson
from environment import Environment
from integersolution import integer_solution
from utils import *
import time

# Import heuristic AHP-mTSP
# Details of the heuristic can be found in "Balanced task allocation by partitioning the
# minmax multiple traveling salesperson" by Isaac Vandermeulen, Roderich Gross, and 
# Andreas Kolling which was published in the proceedings of AAMAS 2019.
sys.path.insert(0, "../ahp_mtsp")
import tsp as ahp

In [2]:
# to save variables on file along the way
# (Notice: the calculations are sometimes very long. We should minimize the risk of losing information)
SAVE_FOLDER = "./saved_results/local_comparison/"
import os
os.makedirs(SAVE_FOLDER, exist_ok=True)

# the three base modules
little_module = "Franch1"  # tsp length: 198, number of nodes: 40
middle_module = "battle_creekhs_1"  # tsp length: 334, number of nodes: 47
big_module = "RomainRolland1"  # tsp length: 438, number of nodes: 80

# the possible types of structures
# - random: modules are chosen randomly between the three base modules (uniform distribution)
# - decreasing: first third of the modules is composed by big modules, second third by middle
#               ones and the third by the little ones
# - increasing: inverse of "decreasing"
STRUCTURE_TYPES = ["random", "decreasing", "increasing"]

# distance between modules [fixed, for simplicity]
CONNECTION_DISTANCE = 20

In [3]:
def random_module_sequence(types, number_of_modules):
    np.random.seed(0) # seed to ensure reproducibility
    
    # being N = number_of_modules and being len(types) == 3
    # first generate a list of (N//3)*3 modules, just replicating the base modules
    # Notice: the list is not long N unless N % 3 == 0
    modules_list = types * (number_of_modules//len(types))
    
    # Now complete the list adding N % 3 modules (choosing randomly from the possible modules)
    missing_modules = np.random.permutation(types)[:number_of_modules%len(types)]
    modules_list.extend(missing_modules)
    
    # Permutate the modules and get the modules sequence
    modules_sequence = np.random.permutation(modules_list)
    return modules_sequence

def decreasing_module_sequence(types, number_of_modules):
    # initizialize to the last type [little] (then overwrite ONLY the lower parts)
    modules_sequence = [max(types)]*number_of_modules
    section_size = int(np.ceil(number_of_modules/len(types)))
    
    for i in range(min(len(types)-1, number_of_modules)):
        modules_sequence[i*section_size:(i+1)*section_size] = [types[i]]*section_size
    
    return modules_sequence
    
def increasing_module_sequence(types, number_of_modules):
    # initizialize to the first type (then overwrite ONLY the lower parts)
    modules_sequence = [min(types)]*number_of_modules
    section_size = int(np.ceil(number_of_modules/len(types)))
    
    for i in range(min(len(types)-1, number_of_modules)):
        modules_sequence[i*section_size:(i+1)*section_size] = [types[len(types)-1-i]]*section_size
    
    return modules_sequence

In [4]:
big_matrix = get_building_matrix(big_module)
middle_matrix = get_building_matrix(middle_module)
little_matrix = get_building_matrix(little_module)

matrices = [big_matrix, middle_matrix, little_matrix]
types = list(range(len(matrices)))

def env_builder(structure_type, number_of_modules):
    modules_sequence = [max(types)]*number_of_modules
    
    if structure_type == "random":
        modules_sequence = random_module_sequence(types, number_of_modules)
    elif structure_type == "decreasing":
        modules_sequence = decreasing_module_sequence(types, number_of_modules)
    elif structure_type == "increasing":
        modules_sequence = increasing_module_sequence(types, number_of_modules)
    else:
        raise ValueError("Allowed values for structure type are: ", STRUCTURE_TYPES)
    
    distances = [CONNECTION_DISTANCE]*(number_of_modules-1)
    
    env = Environment(matrices, modules_sequence, distances)
    
    return env

# Batch calculations for different scenarios

In [5]:
# temporary. The previous cell contains the values used in the paper
structures = ["random", "decreasing", "increasing"]
robots_numbers = [2,5]
modules_numbers = [5,8]

## DO NOT EXECUTE NEXT CELL
The computation could be extremely slow and the results have been saved in files, so there is no need to recompute them unless you want to change parameters. 

**WARNING**: executing the next cell will OVERWRITE the "dump" files in which the results are stored. Be sure to back that up

In [6]:
# be sure to have logger level set to CRITICAL, to avoid slowdowns

smN = []

fred_max_len_list = []
int_max_len_list = []
ahp_max_len_list = []

fred_time_list = []
int_time_list = []
agnostic_int_time_list = []
mat_time_list = []
ahp_time_list = []

for N in modules_numbers:
    for m in robots_numbers:
        for s in structures:
            smN.append( (s,m,N) )
            
            env = env_builder(s,N)

            print("Computing case s={}, m={}, N={}".format(s, m, N))

            ## FREDERICKSON CALCULATION ##
            fred_start_time = time.time()
            fred = Frederickson(env, m)
            fred_tours = fred.calculate()
            fred_max_len, fred_max_tour = max_length(fred_tours, env)
            fred_end_time = time.time()

            fred_time = fred_end_time - fred_start_time

            # speed optimization
            del fred
            del fred_tours
            del fred_max_tour

            ## INTEGER SOLUTION CALCULATION ##
            int_start_time = time.time()
            int_tours = integer_solution(env, m)
            int_max_len, int_max_tour = max_length(int_tours, env)
            int_end_time = time.time()

            int_time = int_end_time - int_start_time
            
            agnostic_int_start_time = time.time()
            agnostic_int_tours = integer_solution(env, m, agnostic_mode=True)
            agnostic_int_max_len, agnostic_int_max_tour = max_length(agnostic_int_tours, env)
            agnostic_int_end_time = time.time()

            agnostic_int_time = agnostic_int_end_time - agnostic_int_start_time

            # speed optimization
            del int_tours, agnostic_int_tours
            del int_max_tour, agnostic_int_max_tour

            ## AHP SOLUTION CALCULATION ##
            ahp_start_time = time.time()
            mat_start_time = time.time()
            mat=env.generate_matrix(for_real=True)
            mat_end_time = time.time()
            weights = [0]*len(mat)
            start_depot = [0]*m
            end_depot = start_depot
            ahp_tours = ahp.minmax_mTSP(mat, weights, start_depot, end_depot)
            ahp_max_len, ahp_max_tour = max_length(ahp_tours, env)
            ahp_end_time = time.time()

            # speed optimization
            del mat
            del ahp_tours
            del env

            mat_time = mat_end_time - mat_start_time
            ahp_time = ahp_end_time - ahp_start_time

            # Save in the lists for later analysis
            fred_max_len_list.append(fred_max_len)
            int_max_len_list.append(int_max_len)
            ahp_max_len_list.append(ahp_max_len)

            fred_time_list.append(fred_time)
            int_time_list.append(int_time)
            agnostic_int_time_list.append(agnostic_int_time)
            mat_time_list.append(mat_time)
            ahp_time_list.append(ahp_time)

            save_variables(fred_max_len_list, "fred_max_len_list.dump", folder=SAVE_FOLDER)
            save_variables(int_max_len_list, "int_max_len_list.dump", folder=SAVE_FOLDER)
            save_variables(ahp_max_len_list, "ahp_max_len_list.dump", folder=SAVE_FOLDER)
            save_variables(fred_time_list, "fred_time_list.dump", folder=SAVE_FOLDER)
            save_variables(int_time_list, "int_time_list.dump", folder=SAVE_FOLDER)
            save_variables(agnostic_int_time_list, "agnostic_int_time_list.dump", folder=SAVE_FOLDER)
            save_variables(mat_time_list, "mat_time_list.dump", folder=SAVE_FOLDER)
            save_variables(ahp_time_list, "ahp_time_list.dump", folder=SAVE_FOLDER)
            save_variables(smN, "smN.dump", folder=SAVE_FOLDER)

Computing case s=random, m=2, N=5
Computing case s=decreasing, m=2, N=5
Computing case s=increasing, m=2, N=5
Computing case s=random, m=5, N=5
Computing case s=decreasing, m=5, N=5
Computing case s=increasing, m=5, N=5
Computing case s=random, m=2, N=8
Computing case s=decreasing, m=2, N=8
Computing case s=increasing, m=2, N=8
Computing case s=random, m=5, N=8
Computing case s=decreasing, m=5, N=8
Computing case s=increasing, m=5, N=8


In [ ]:
# needed on aws, to know that the process has ended properly in case ssh connection is lost
save_variables("finished", "finished.dump", folder=SAVE_FOLDER)

In [7]:
for i in range(len(smN)):
    print("Case (structure, number of robots, number of modules):", smN[i])
    print("len fred:", fred_max_len_list[i])
    print("len int:", int_max_len_list[i])
    print("len ahp:", ahp_max_len_list[i])
    print("time fred:", round(fred_time_list[i], 3))
    print("time int:", round(int_time_list[i], 3))
    print("agnostic time int:", round(agnostic_int_time_list[i], 3))
    print("time mat:", round(mat_time_list[i], 3))
    print("time ahp:", round(ahp_time_list[i], 3))
    print()

Case (structure, number of robots, number of modules): ('random', 2, 5)
len fred: 1753.11759608
len int: 945.34944981
len ahp: 848.99639146
time fred: 0.922
time int: 0.036
agnostic time int: 0.048
time mat: 0.654
time ahp: 5.812

Case (structure, number of robots, number of modules): ('decreasing', 2, 5)
len fred: 1932.23813012
len int: 1025.34944981
len ahp: 1090.25682595
time fred: 1.081
time int: 0.036
agnostic time int: 0.064
time mat: 0.763
time ahp: 8.363

Case (structure, number of robots, number of modules): ('increasing', 2, 5)
len fred: 1605.93182841
len int: 931.748977807
len ahp: 865.109297076
time fred: 0.864
time int: 0.042
agnostic time int: 0.056
time mat: 0.597
time ahp: 5.666

Case (structure, number of robots, number of modules): ('random', 5, 5)
len fred: 801.916619364
len int: 598.208196048
len ahp: 447.494754825
time fred: 0.84
time int: 0.036
agnostic time int: 0.048
time mat: 0.608
time ahp: 7.729

Case (structure, number of robots, number of modules): ('decrea

# Calculations for `m > n`

## DO NOT EXECUTE NEXT CELL
The computation could be slow and the results have been saved in files, so there is no need to recompute them unless you want to change parameters. 

**WARNING**: executing the next cell will OVERWRITE the "dump" files in which the results are stored. Be sure to back that up

In [8]:
# mgn = m greater than n

mgn_N = 10 # N is set to 10 for all the next experiments
mgn_robots_numbers = list(range(1,31))

mgn_SAVE_FOLDER = "./saved_results/mgn_local_comparison/"
os.makedirs(mgn_SAVE_FOLDER, exist_ok=True)

In [9]:
# temporary. The previous cell contains the values used in the paper
mgn_N = 3
mgn_robots_numbers = [2,5]

In [10]:
# be sure to have logger level set to CRITICAL, to avoid slowdowns

m_values = []

mgn_fred_max_len_list = []
mgn_int_max_len_list = []
mgn_ahp_max_len_list = []

for m in mgn_robots_numbers:
    m_values.append(m)
    structure = "random"
    env = env_builder(structure, mgn_N)

    print("Computing case m={}".format(m))
    
    
    print("\tFrederickson")
    ## FREDERICKSON CALCULATION ##
    mgn_fred = Frederickson(env, m)
    mgn_fred_tours = mgn_fred.calculate()
    mgn_fred_max_len, mgn_fred_max_tour = max_length(mgn_fred_tours, env)

    # speed optimization
    del mgn_fred
    del mgn_fred_tours
    del mgn_fred_max_tour
    
    print("\tInteger solution")
    ## INTEGER SOLUTION CALCULATION ##
    mgn_int_tours = integer_solution(env, m)
    mgn_int_max_len, mgn_int_max_tour = max_length(mgn_int_tours, env)

    # speed optimization
    del mgn_int_tours
    del mgn_int_max_tour
    
    print("\tAHP")
    ## AHP SOLUTION CALCULATION ##
    mat = env.generate_matrix(for_real=True)
    weights = [0]*len(mat)
    start_depot = [0]*m
    end_depot = start_depot
    mgn_ahp_tours = ahp.minmax_mTSP(mat, weights, start_depot, end_depot)
    mgn_ahp_max_len, mgn_ahp_max_tour = max_length(mgn_ahp_tours, env)

    # speed optimization
    del mat
    del mgn_ahp_tours
    del env


    # Save in the lists for later analysis
    mgn_fred_max_len_list.append(mgn_fred_max_len)
    mgn_int_max_len_list.append(mgn_int_max_len)
    mgn_ahp_max_len_list.append(mgn_ahp_max_len)

    save_variables(mgn_fred_max_len_list, "mgn_fred_max_len_list.dump", folder=mgn_SAVE_FOLDER)
    save_variables(mgn_int_max_len_list, "mgn_int_max_len_list.dump", folder=mgn_SAVE_FOLDER)
    save_variables(mgn_ahp_max_len_list, "mgn_ahp_max_len_list.dump", folder=mgn_SAVE_FOLDER)
    save_variables(m_values, "m_values.dump", folder=mgn_SAVE_FOLDER)

# needed on aws, to know that the process has ended properly in case ssh connection is lost
save_variables("mgn_finished", "mgn_finished.dump", folder=mgn_SAVE_FOLDER)

Computing case m=2
	Frederickson
	Integer solution
	AHP
Computing case m=5
	Frederickson
	Integer solution
	AHP
